<a href="https://colab.research.google.com/github/Namsik-Yoon/pygcn/blob/master/experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Namsik-Yoon/pygcn.git

fatal: destination path 'pygcn' already exists and is not an empty directory.


In [2]:
import os
import sys
os.chdir('/content/pygcn')
sys.path.insert(1, '/content/pygcn')

In [3]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing pygcn.egg-info/PKG-INFO
writing dependency_links to pygcn.egg-info/dependency_links.txt
writing requirements to pygcn.egg-info/requires.txt
writing top-level names to pygcn.egg-info/top_level.txt
adding license file 'LICENCE'
writing manifest file 'pygcn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying pygcn/models.py -> build/lib/pygcn
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pygcn
copying build/lib/pygcn/__init__.py -> build/bdist.linux-x86_64/egg/pygcn
copying build/lib/pygcn/train.py -> build/bdist.linux-x86_64/egg/pygcn
copying build/lib/pygcn/layers.py -> build/bdist.linux-x86_64/egg/pygcn
copying build/lib/pygcn/utils.py -> build/bdist.linux-x86_64/egg/pygcn
copying build/lib/pygcn/models.py -> build/bdist.linux-x86_64/egg/pygcn
byte-compiling build/bdist.linux-x86_64/egg/pygcn/__init__.py to __init__.cpython-37.p

In [39]:
import random
import numpy as np
import scipy.sparse as sp
import torch

In [5]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [6]:
path="/content/pygcn/data/cora/"
dataset="cora"

In [7]:
## content 데이터 
## [논문의 아이디,각 논문의 feature, 논문의 장르]
idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
idx_features_labels

array([['31336', '0', '0', ..., '0', '0', 'Neural_Networks'],
       ['1061127', '0', '0', ..., '0', '0', 'Rule_Learning'],
       ['1106406', '0', '0', ..., '0', '0', 'Reinforcement_Learning'],
       ...,
       ['1128978', '0', '0', ..., '0', '0', 'Genetic_Algorithms'],
       ['117328', '0', '0', ..., '0', '0', 'Case_Based'],
       ['24043', '0', '0', ..., '0', '0', 'Neural_Networks']],
      dtype='<U22')

In [8]:
## 각 논문의 feature
features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
## 논문의 장르(onehot)
labels = encode_onehot(idx_features_labels[:, -1])

In [9]:
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
idx_map = {j: i for i, j in enumerate(idx)} ## 논문 id 맵핑 dict

In [10]:
## cite 데이터
## [참조된 논문, 참조한 논문]
edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                dtype=np.int32)
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                    dtype=np.int32).reshape(edges_unordered.shape)

## cite 데이터로 directed graph의 adj matrix 생성
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                    shape=(labels.shape[0], labels.shape[0]),
                    dtype=np.float32)
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

In [11]:
features = normalize(features)
adj = normalize(adj + sp.eye(adj.shape[0]))

In [12]:
features = torch.FloatTensor(np.array(features.todense()))
labels = torch.LongTensor(np.where(labels)[1])
adj = sparse_mx_to_torch_sparse_tensor(adj)

In [13]:
features,features.shape

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]), torch.Size([2708, 1433]))

In [14]:
labels,labels.shape

(tensor([5, 1, 0,  ..., 2, 4, 5]), torch.Size([2708]))

In [15]:
adj,adj.shape

(tensor(indices=tensor([[   0,    8,   14,  ..., 1389, 2344, 2707],
                        [   0,    0,    0,  ..., 2707, 2707, 2707]]),
        values=tensor([0.1667, 0.1667, 0.0500,  ..., 0.2000, 0.5000, 0.2500]),
        size=(2708, 2708), nnz=13264, layout=torch.sparse_coo),
 torch.Size([2708, 2708]))

In [16]:
idx_train = torch.LongTensor(range(1200))
idx_val = torch.LongTensor(range(1200,1500))
idx_test = torch.LongTensor(range(1500,2708))

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim

random_seed = 2021
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [49]:
class GCN_layer(nn.Module):
    def __init__(self, in_features, out_features, A):
        super(GCN_layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.A = A
        self.fc = nn.Linear(in_features, out_features)
        
    def forward(self, X):
        return self.fc(torch.spmm(self.A, X)) #이웃 정보 종합

class GCN(nn.Module):
    def __init__(self, num_feature, num_class, A):
        super(GCN, self).__init__()

        self.feature_extractor = nn.Sequential(
                                    GCN_layer(num_feature, 16, A),
                                    nn.ReLU(),
                                    GCN_layer(16, num_class, A)
                                )
        
    def forward(self, X):
        return self.feature_extractor(X)

In [50]:
class FCN(nn.Module):
    def __init__(self, num_feature, num_class):
        super(FCN, self).__init__()

        self.feature_extractor = nn.Sequential(
                                    nn.Linear(num_feature, 16),
                                    nn.ReLU(),
                                    nn.Linear(16, num_class)
                                )

    def forward(self, x):
        return self.feature_extractor(x)

In [51]:
def train(model, Loss, optimizer, num_epochs):
  train_loss_arr = []
  test_loss_arr = []

  best_test_loss = 99999999
  best_ACC = 0
  early_stop, early_stop_max = 0., 10.

  for epoch in range(num_epochs):

    # Forward Pass
    model.train()
    output = model(features)
    train_loss = criterion(output[idx_train], labels[idx_train])

    # Backward and optimize
    train_loss.backward()
    optimizer.step()
        
    train_loss_arr.append(train_loss.data)
    
    if epoch % 20 == 0:
        model.eval()
        
        output = model(features)
        val_loss = criterion(output[idx_val], labels[idx_val])
        test_loss = criterion(output[idx_test], labels[idx_test])
        
        val_acc = accuracy(output[idx_val], labels[idx_val])
        test_acc = accuracy(output[idx_test], labels[idx_test])
        
        test_loss_arr.append(test_loss)
        
        if best_ACC < val_acc:
            best_ACC = val_acc
            early_stop = 0
            final_ACC = test_acc
            print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}, Test ACC: {:.4f} *'.format(epoch, num_epochs, train_loss.data, test_loss, test_acc))
        else:
            early_stop += 1

            print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}, Test ACC: {:.4f}'.format(epoch, num_epochs, train_loss.data, test_loss, test_acc))

    if early_stop >= early_stop_max:
        break
        
  print("Final Accuracy::", final_ACC)

In [52]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [53]:
# FCN 학습 돌려서 epoch에 따른 Loss 확인
model = FCN(features.size(1) , labels.unique().size(0))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)

train(model, criterion, optimizer, 1000)


# GCN 학습 돌려서 epoch에 따른 Loss 확인
model = GCN(features.size(1) , labels.unique().size(0), adj)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)

train(model, criterion, optimizer, 1000)

Epoch [0/1000], Train Loss: 1.9693, Test Loss: 1.9561, Test ACC: 0.0952 *
Epoch [20/1000], Train Loss: 1.7135, Test Loss: 1.7701, Test ACC: 0.2864 *
Epoch [40/1000], Train Loss: 1.4001, Test Loss: 1.6211, Test ACC: 0.3187 *
Epoch [60/1000], Train Loss: 1.2610, Test Loss: 1.5977, Test ACC: 0.4536 *
Epoch [80/1000], Train Loss: 0.6459, Test Loss: 1.2421, Test ACC: 0.5381 *
Epoch [100/1000], Train Loss: 1.1286, Test Loss: 2.2688, Test ACC: 0.4040
Epoch [120/1000], Train Loss: 0.7650, Test Loss: 1.7546, Test ACC: 0.5141
Epoch [140/1000], Train Loss: 1.0105, Test Loss: 2.0794, Test ACC: 0.4652
Epoch [160/1000], Train Loss: 1.1740, Test Loss: 3.0651, Test ACC: 0.3601
Epoch [180/1000], Train Loss: 0.7834, Test Loss: 2.8174, Test ACC: 0.4636
Epoch [200/1000], Train Loss: 0.8743, Test Loss: 2.9947, Test ACC: 0.4743
Epoch [220/1000], Train Loss: 0.5856, Test Loss: 2.6093, Test ACC: 0.5455
Epoch [240/1000], Train Loss: 0.7827, Test Loss: 3.2212, Test ACC: 0.5397
Epoch [260/1000], Train Loss: 0.94